In [13]:
import pandas as pd
import numpy as np

# 원본 데이터 불러오기
amazon_raw = pd.read_csv("../01_data/01_raw/amazon_delivery_raw.csv")
amazon_raw.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [14]:
# 필요없는 열 제거
df = amazon_raw.drop(columns = ["Order_ID", "Agent_Age", "Agent_Rating", "Category"])
df.head()

,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time
0,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120
1,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165
2,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130
3,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105
4,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150


In [15]:
# 날짜 기준으로 정렬
df["Order_Date"] = pd.to_datetime(df["Order_Date"])
df = df.sort_values(by = "Order_Date").reset_index(drop = True)
df.head()

,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time
0,15.544419,73.755736,15.604419,73.815736,2022-02-11,18:40:00,18:55:00,Windy,Medium,scooter,Urban,105
1,0.000000,0.000000,0.050000,0.050000,2022-02-11,19:25:00,19:30:00,Fog,Jam,van,Metropolitian,150
2,22.551084,88.354127,22.611084,88.414127,2022-02-11,18:15:00,18:30:00,Sandstorms,Medium,motorcycle,Metropolitian,85
3,30.892860,75.822199,30.942860,75.872199,2022-02-11,23:55:00,00:00:00,Sandstorms,Low,motorcycle,Metropolitian,160
4,10.020683,76.310631,10.060683,76.350631,2022-02-11,13:40:00,13:55:00,Sandstorms,High,motorcycle,Metropolitian,225


In [16]:
# 날짜 열 제거
df = df.drop(columns = ["Order_Date"])
df.head()

,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time
0,15.544419,73.755736,15.604419,73.815736,18:40:00,18:55:00,Windy,Medium,scooter,Urban,105
1,0.000000,0.000000,0.050000,0.050000,19:25:00,19:30:00,Fog,Jam,van,Metropolitian,150
2,22.551084,88.354127,22.611084,88.414127,18:15:00,18:30:00,Sandstorms,Medium,motorcycle,Metropolitian,85
3,30.892860,75.822199,30.942860,75.872199,23:55:00,00:00:00,Sandstorms,Low,motorcycle,Metropolitian,160
4,10.020683,76.310631,10.060683,76.350631,13:40:00,13:55:00,Sandstorms,High,motorcycle,Metropolitian,225


In [17]:
# 위도와 경도를 인자로 받는 함수 하버사인 정의
def haversine(lat1, lon1, lat2, lon2):
    
    # 삼각함수 sin, cos은 라디안만 받으므로 위도와 경도의 값을 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 두 점 사이의 위도, 경도 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 거리 = 반지름 * 각도인데 위도 경도로는 각도를 알 수 없으므로 위도 경도만으로 각도를 알 수 있게 하기 위한 중간 값 계산
    a = (np.sin(dlat/2) ** 2 + np.cos(lat1) * np.cos(lat2) *np.sin(dlon/2)**2)

    # a = sin^2(theta/2)이므로 theta = 2 * arcsin(sqrt(a))
    theta = 2 * np.arcsin(np.sqrt(a))

    # 지구 반지름
    R = 6371

    # 거리 = 반지름 * 각도
    return R * theta
    
# 각 행마다 출발지와 도착지의 위도, 경도를 가져와 하버사인 공식을 이용해 두 지점 사이의 거리를 계산
# axis = 1 이므로 행 기준으로 함수가 적용됨
df["Distance"] = df.apply(lambda row: haversine(row["Store_Latitude"], row["Store_Longitude"], row["Drop_Latitude"], row["Drop_Longitude"]), 
axis = 1)
df.head()

,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Distance
0,15.544419,73.755736,15.604419,73.815736,18:40:00,18:55:00,Windy,Medium,scooter,Urban,105,9.263603
1,0.000000,0.000000,0.050000,0.050000,19:25:00,19:30:00,Fog,Jam,van,Metropolitian,150,7.862668
2,22.551084,88.354127,22.611084,88.414127,18:15:00,18:30:00,Sandstorms,Medium,motorcycle,Metropolitian,85,9.080742
3,30.892860,75.822199,30.942860,75.872199,23:55:00,00:00:00,Sandstorms,Low,motorcycle,Metropolitian,160,7.325376
4,10.020683,76.310631,10.060683,76.350631,13:40:00,13:55:00,Sandstorms,High,motorcycle,Metropolitian,225,6.242152


In [18]:
# 위치 정보 계산이 완료되었으므로 위도와 경도 열 제거
df = df.drop(columns = ["Store_Latitude", "Store_Longitude", "Drop_Latitude", "Drop_Longitude"])
df.head()

,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Distance
0,18:40:00,18:55:00,Windy,Medium,scooter,Urban,105,9.263603
1,19:25:00,19:30:00,Fog,Jam,van,Metropolitian,150,7.862668
2,18:15:00,18:30:00,Sandstorms,Medium,motorcycle,Metropolitian,85,9.080742
3,23:55:00,00:00:00,Sandstorms,Low,motorcycle,Metropolitian,160,7.325376
4,13:40:00,13:55:00,Sandstorms,High,motorcycle,Metropolitian,225,6.242152


In [19]:
# 문자열로 된 주문 시각과 픽업 시각을 datetime 형식으로 변환
# (시:분:초 포맷, 변환 불가한 값은 NaT 처리)
df["Order_Time_dt"] = pd.to_datetime(df["Order_Time"], format="%H:%M:%S", errors="coerce")
df["Pickup_Time_dt"] = pd.to_datetime(df["Pickup_Time"], format="%H:%M:%S", errors="coerce")

# 하루를 0~1439 범위의 분 단위 숫자로 변환 [시각 = (시 * 60 + 분)]
df["Order_Minute"] = (df["Order_Time_dt"].dt.hour * 60 + df["Order_Time_dt"].dt.minute)
df["Pickup_Minute"] = (df["Pickup_Time_dt"].dt.hour * 60 + df["Pickup_Time_dt"].dt.minute)

# 픽업 분 - 주문 분으로 픽업 대기 시간 계산
df["Pickup_Wait_Time"] = df["Pickup_Minute"] - df["Order_Minute"]

# 자정을 넘긴 경우(음수 값)는 다음 날로 간주하여 1440분 보정
df.loc[df["Pickup_Wait_Time"] < 0, "Pickup_Wait_Time"] += 1440
df.head()

,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Distance,Order_Time_dt,Pickup_Time_dt,Order_Minute,Pickup_Minute,Pickup_Wait_Time
0,18:40:00,18:55:00,Windy,Medium,scooter,Urban,105,9.263603,1900-01-01 18:40:00,1900-01-01 18:55:00,1120.0,1135,15.0
1,19:25:00,19:30:00,Fog,Jam,van,Metropolitian,150,7.862668,1900-01-01 19:25:00,1900-01-01 19:30:00,1165.0,1170,5.0
2,18:15:00,18:30:00,Sandstorms,Medium,motorcycle,Metropolitian,85,9.080742,1900-01-01 18:15:00,1900-01-01 18:30:00,1095.0,1110,15.0
3,23:55:00,00:00:00,Sandstorms,Low,motorcycle,Metropolitian,160,7.325376,1900-01-01 23:55:00,1900-01-01 00:00:00,1435.0,0,5.0
4,13:40:00,13:55:00,Sandstorms,High,motorcycle,Metropolitian,225,6.242152,1900-01-01 13:40:00,1900-01-01 13:55:00,820.0,835,15.0


In [20]:
# 시간 정보 계산이 완료되었으므로 필요한 열만 선택
df = df[["Distance", "Pickup_Wait_Time", "Weather", "Traffic", "Vehicle", "Area", "Delivery_Time"]].copy()
df.head()

,Distance,Pickup_Wait_Time,Weather,Traffic,Vehicle,Area,Delivery_Time
0,9.263603,15.0,Windy,Medium,scooter,Urban,105
1,7.862668,5.0,Fog,Jam,van,Metropolitian,150
2,9.080742,15.0,Sandstorms,Medium,motorcycle,Metropolitian,85
3,7.325376,5.0,Sandstorms,Low,motorcycle,Metropolitian,160
4,6.242152,15.0,Sandstorms,High,motorcycle,Metropolitian,225


In [21]:
# 범주형 변수 정리를 위해 문자열 양쪽 공백 제거
# (공백 때문에 동일 범주가 다른 값으로 인식되는 것 방지)
df["Traffic"] = df["Traffic"].astype(str).str.strip()
df["Weather"] = df["Weather"].astype(str).str.strip()
df["Vehicle"] = df["Vehicle"].astype(str).str.strip()
df["Area"] = df["Area"].astype(str).str.strip()

# 날씨 변수 원-핫 인코딩
# 기준 범주(Sunny)는 제거하여 다중공선성 방지
weather_dummies = pd.get_dummies(df["Weather"], prefix = "Weather")

# 범주형 변수 원-핫 인코딩 결과에서 기준 범주를 제거
# (모든 범주를 유지하면 각 행에서 더미 변수 합이 항상 1이 되어 하나의 컬럼이 나머지 컬럼들로 완전히 표현되는 다중공선성 발생)
weather_dummies = weather_dummies.drop(columns = ["Weather_Sunny"])

# 교통 상황 변수 원-핫 인코딩
# 기준 범주(Low) 제거
traffic_dummies = pd.get_dummies(df["Traffic"], prefix = "Traffic")
traffic_dummies = traffic_dummies.drop(columns = ["Traffic_Low"])

# 운송 수단 변수 원-핫 인코딩
# 기준 범주(motorcycle) 제거
vehicle_dummies = pd.get_dummies(df["Vehicle"], prefix="Vehicle")
vehicle_dummies = vehicle_dummies.drop(columns=["Vehicle_motorcycle"])

# 지역 유형 변수 원-핫 인코딩
# 기준 범주(Metropolitian) 제거
area_dummies = pd.get_dummies(df["Area"], prefix="Area")
area_dummies = area_dummies.drop(columns=["Area_Metropolitian"])

# 생성된 더미 변수를 행 기준으로 기존 데이터프레임에 병합
df = pd.concat([df, weather_dummies, traffic_dummies, vehicle_dummies, area_dummies], axis = 1)
df.head()

,Distance,Pickup_Wait_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Weather_Cloudy,Weather_Fog,Weather_Sandstorms,...,Traffic_High,Traffic_Jam,Traffic_Medium,Traffic_NaN,Vehicle_bicycle,Vehicle_scooter,Vehicle_van,Area_Other,Area_Semi-Urban,Area_Urban
0,9.263603,15.0,Windy,Medium,scooter,Urban,105,False,False,False,...,False,False,True,False,False,True,False,False,False,True
1,7.862668,5.0,Fog,Jam,van,Metropolitian,150,False,True,False,...,False,True,False,False,False,False,True,False,False,False
2,9.080742,15.0,Sandstorms,Medium,motorcycle,Metropolitian,85,False,False,True,...,False,False,True,False,False,False,False,False,False,False
3,7.325376,5.0,Sandstorms,Low,motorcycle,Metropolitian,160,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,6.242152,15.0,Sandstorms,High,motorcycle,Metropolitian,225,False,False,True,...,True,False,False,False,False,False,False,False,False,False


In [22]:
# 환경 정보 계산이 완료되었으므로 환경 정보 원본 열을 제거
df = df.drop(columns = ["Weather", "Traffic", "Vehicle", "Area"])
df.head(50000)

,Distance,Pickup_Wait_Time,Delivery_Time,Weather_Cloudy,Weather_Fog,Weather_Sandstorms,Weather_Stormy,Weather_Windy,Weather_nan,Traffic_High,Traffic_Jam,Traffic_Medium,Traffic_NaN,Vehicle_bicycle,Vehicle_scooter,Vehicle_van,Area_Other,Area_Semi-Urban,Area_Urban
0,9.263603,15.0,105,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True
1,7.862668,5.0,150,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False
2,9.080742,15.0,85,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False
3,7.325376,5.0,160,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,6.242152,15.0,225,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43734,10.590448,10.0,215,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False
43735,12.419710,5.0,31,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
43736,12.257491,10.0,75,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
43737,17.098754,5.0,18,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False


In [23]:
# 환경 정보가 결측치인 열의 값이 1인 행을 제거한 후, 인덱스 번호와 열 이름 수정
df = df[~((df["Weather_nan"] == 1) | (df["Traffic_NaN"] == 1))]
df = df.drop(columns=["Weather_nan", "Traffic_NaN"])
df = df.dropna().reset_index(drop=True)
df = df.rename(columns={"Area_Semi-Urban":"Area_Semi_Urban"})
df.head(50000)

,Distance,Pickup_Wait_Time,Delivery_Time,Weather_Cloudy,Weather_Fog,Weather_Sandstorms,Weather_Stormy,Weather_Windy,Traffic_High,Traffic_Jam,Traffic_Medium,Vehicle_bicycle,Vehicle_scooter,Vehicle_van,Area_Other,Area_Semi_Urban,Area_Urban
0,9.263603,15.0,105,False,False,False,False,True,False,False,True,False,True,False,False,False,True
1,7.862668,5.0,150,False,True,False,False,False,False,True,False,False,False,True,False,False,False
2,9.080742,15.0,85,False,False,True,False,False,False,False,True,False,False,False,False,False,False
3,7.325376,5.0,160,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,6.242152,15.0,225,False,False,True,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43643,10.590448,10.0,215,False,True,False,False,False,False,False,True,False,False,False,False,False,False
43644,12.419710,5.0,31,False,False,False,False,False,False,True,False,False,False,False,False,False,False
43645,12.257491,10.0,75,False,False,False,False,True,False,False,False,False,False,False,False,False,True
43646,17.098754,5.0,18,False,False,False,False,False,False,False,True,False,True,False,False,False,False


In [24]:
# 전처리된 데이터 저장
df.to_csv("../data/amazon_delivery_processed.csv", index=False, encoding="utf-8-sig")

In [25]:
# 타깃 변수(Delivery_Time)를 분리하여 입력 변수(X)와 정답 변수(y) 생성
X = df.drop(columns=["Delivery_Time"])
y = df["Delivery_Time"]

In [26]:
from sklearn.model_selection import train_test_split

# 전체 데이터를 학습용(train)과 평가용(test)으로 분리
# 전체 데이터의 20%를 테스트용으로 사용하고(test_size=0.2) 데이터 분할 결과를 항상 동일하게 유지(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [27]:
# 학습 데이터 저장
X_train.to_csv("../data/amazon_delivery_X_train.csv", index=False, encoding="utf-8-sig")
X_test.to_csv("../data/amazon_delivery_X_test.csv", index=False, encoding="utf-8-sig")

# 평가 데이터 저장
y_train.to_csv("../data/amazon_delivery_y_train.csv", index=False, encoding="utf-8-sig")
y_test.to_csv("../data/amazon_delivery_y_test.csv", index=False, encoding="utf-8-sig")